In [1]:
import datetime
import numpy as np
import os
import pandas as pd
import spektral
import tensorflow as tf
import tensorflow_addons as tfa

from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.engine.input_layer import InputLayer
from keras.models import load_model
from keras.layers import Activation
from keras import backend as K
from scipy.sparse import csr_array, csr_matrix, load_npz

tf.keras.backend.set_floatx('float32')

2023-03-15 07:33:02.843329: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-15 07:33:03.461458: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-15 07:33:06.418000: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-15 07:33:06.420598: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

In [2]:
class GraphDataset(spektral.data.Dataset):
    def __init__(self, path, **kwargs):
        self.data_path = path
        super().__init__(**kwargs)

    def read(self):
        output = []
        for i in range(int(len(os.listdir(self.data_path))/2)):
            graph = np.load(self.data_path + "graph_{}.npz".format(i))
            adjacency = load_npz(self.data_path + "adjacency_{}.npz".format(i))
            output.append(spektral.data.graph.Graph(x=graph['x'],
                                                    a=adjacency,
                                                    y=graph['y']))
        return output

In [3]:
train_data = GraphDataset("dataset/train/")
test_data = GraphDataset("dataset/test/")
val_data = GraphDataset("dataset/validation/")

In [4]:
def get_generator(dataset):
    def data_generator():
        for graph in dataset:
            yield graph.x[0,:6], graph.y
    return data_generator

In [5]:
train_ds = tf.data.Dataset.from_generator(get_generator(train_data), 
                                          output_signature=(tf.TensorSpec(shape=6, dtype=tf.float32),
                                                            tf.TensorSpec(shape=(), dtype=tf.float32)))
val_ds = tf.data.Dataset.from_generator(get_generator(val_data),
                                        output_signature=(tf.TensorSpec(shape=6, dtype=tf.float32),
                                                            tf.TensorSpec(shape=(), dtype=tf.float32)))
test_ds = tf.data.Dataset.from_generator(get_generator(test_data),
                                         output_signature=(tf.TensorSpec(shape=6, dtype=tf.float32),
                                                            tf.TensorSpec(shape=(), dtype=tf.float32)))

train_ds = train_ds.batch(64)
val_ds = val_ds.batch(64)
test_ds = test_ds.batch(64)

2023-03-15 07:35:32.151046: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-15 07:35:32.151107: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-15 07:35:32.151153: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (icme-gpu1): /proc/driver/nvidia/version does not exist
2023-03-15 07:35:32.158450: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [27]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(12, activation="relu"),
    tf.keras.layers.Dense(12, activation="relu"),
    tf.keras.layers.Dense(6, activation="relu"),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(1, activation="sigmoid"),
])

In [28]:
model.compile(optimizer=tf.optimizers.Adam(learning_rate=1e-3),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.losses.MeanSquaredError(), tf.keras.metrics.MeanAbsoluteError()])

In [29]:
log_dir = "logs/baseline_rightencoding/"
log_dir_train = log_dir + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard = tf.keras.callbacks.TensorBoard(log_dir=log_dir_train, histogram_freq=1)
early_stopping = EarlyStopping(monitor='val_loss',
                               restore_best_weights=True, patience=50,
                               verbose=0, mode='min')

In [30]:
model.fit(train_ds,
          epochs=50,
          validation_data=val_ds,
          callbacks=[tensorboard, early_stopping],
          verbose=1)

Epoch 1/50
182/182 [==============================] - 16s 58ms/step - loss: 0.4509 - mean_squared_error: 0.0321 - mean_absolute_error: 0.1194 - val_loss: 0.4240 - val_mean_squared_error: 0.0225 - val_mean_absolute_error: 0.0927
Epoch 2/50
182/182 [==============================] - 10s 55ms/step - loss: 0.4218 - mean_squared_error: 0.0218 - mean_absolute_error: 0.0915 - val_loss: 0.4190 - val_mean_squared_error: 0.0213 - val_mean_absolute_error: 0.0867
Epoch 3/50
182/182 [==============================] - 8s 46ms/step - loss: 0.4183 - mean_squared_error: 0.0210 - mean_absolute_error: 0.0891 - val_loss: 0.4164 - val_mean_squared_error: 0.0206 - val_mean_absolute_error: 0.0842
Epoch 4/50
182/182 [==============================] - 8s 46ms/step - loss: 0.4162 - mean_squared_error: 0.0205 - mean_absolute_error: 0.0878 - val_loss: 0.4154 - val_mean_squared_error: 0.0204 - val_mean_absolute_error: 0.0834
Epoch 5/50
182/182 [==============================] - 10s 52ms/step - loss: 0.4148 - mean_

In [31]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 12)                84        
                                                                 
 dense_13 (Dense)            (None, 12)                156       
                                                                 
 dense_14 (Dense)            (None, 6)                 78        
                                                                 
 dense_15 (Dense)            (None, 128)               896       
                                                                 
 dense_16 (Dense)            (None, 128)               16512     
                                                                 
 dense_17 (Dense)            (None, 1)                 129       
                                                                 
Total params: 17,855
Trainable params: 17,855
Non-trai

In [32]:
model.save_weights(log_dir + "model_weights")